<a href="https://colab.research.google.com/github/ShenetzK/ShenetzK/blob/main/CivilioBel_RegressionNet_Git.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# Работа с массивами данных
import numpy as np

# Работа с табличными данными
import pandas as pd

# Функции-утилиты для работы с категориальными данными
from tensorflow.keras import utils

# Класс для конструирования последовательной модели нейронной сети
from tensorflow.keras.models import Sequential, Model

# Основные слои
from tensorflow.keras.layers import Dense, Dropout, SpatialDropout1D, BatchNormalization, Embedding, Flatten, Activation, Input, concatenate
from tensorflow.keras.layers import SimpleRNN, GRU, LSTM, Bidirectional, Conv1D, MaxPooling1D, GlobalMaxPooling1D

# Оптимизаторы
from tensorflow.keras.optimizers import Adam, Adadelta, SGD, Adagrad, RMSprop

# Токенизатор для преобразование текстов в последовательности
from tensorflow.keras.preprocessing.text import Tokenizer

# Масштабирование данных
from sklearn.preprocessing import StandardScaler

# Загрузка датасетов из облака google
import gdown

# Регулярные выражения
import re

# Отрисовка графиков
import matplotlib.pyplot as plt

# Метрики для расчета ошибок
from sklearn.metrics import mean_squared_error, mean_absolute_error

%matplotlib inline

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:

# Загрузка датасета
df = pd.read_csv('/content/drive/MyDrive/SivilioBel_all_info_ver1.csv', index_col=0)

# Вывод первых нескольких строк датасета для проверки
display(df.head(1))
print(df.shape)

,Тип события,Вакансия,Компания,Кандидат,Дата события,Примечание события,Скилы для вакансии,Основной навык для вакансии,Зарплата для вакансии,Уровень для вакансии,...,Зарплата кандидата,Опыт работы кандидата,График кандидата,Регион кандидата,Место кандидата,Профессия кандидата,Английский кандидата,Примечание кандидата,Ссылка на резюме,Файл с резюме
№,,,,,,,,,,,,,,,,,,,,,
1,Оффер,ML Lead,PersonalInvest - Indata Labs,Бакаев Евгений,01.04.2024,01.04 - на рассмотрение 07.04 - интервью Бакае...,"AWS, Azure, ETL, Machine Learning, python",Machine Learning,8000-12000 в месяц,Head of,...,NaN,15.0,Полный день,Варшава,Удаленная,"Data scientist, Machine Learning Engineer",B2,NaN,NaN,NaN


(6212, 26)


In [12]:
# Замена концов строк на пробелы, удаление символа с кодом 0xA0
# обрезка краевых пробелов, приведение к нижнему регистру

def purify(x):
    if isinstance(x, str):                # Если значение - строка:
        x = x.replace('\n', ' ').replace('\xa0', '').strip().lower()
    return x

In [13]:
# Настройка номеров столбцов

COL_IVENT_TYPE     = df.columns.get_loc('Тип события')
COL_VACANCY     = df.columns.get_loc('Вакансия')
COL_COMPANY     = df.columns.get_loc('Компания')
COL_PERSON     = df.columns.get_loc('Кандидат')
COL_IVENT_DATE  = df.columns.get_loc('Дата события')
COL_IVENT_COMM  = df.columns.get_loc('Примечание события')
COL_SKILLS     = df.columns.get_loc('Скилы для вакансии')
COL_BASE_SKILL     = df.columns.get_loc('Основной навык для вакансии')
COL_VAC_SALARY      = df.columns.get_loc('Зарплата для вакансии')
COL_LEVEL_VACANCY     = df.columns.get_loc('Уровень для вакансии') #Уровень, требуемый для вакансии
# COL_POS_SEEK    = df.columns.get_loc('Ищет работу на должность:')
COL_ENG_VAC       = df.columns.get_loc('Английский для вакансии')
COL_EMPL_VAC      = df.columns.get_loc('Место для вакансии')
COL_SCHED_VAC     = df.columns.get_loc('График для вакансии')
COL_LEVEL_PERS    = df.columns.get_loc('Уровень кандидата')
COL_SKILLS_PERS   = df.columns.get_loc('Скилы кандидата')
COL_BASE_SKILL_PERS = df.columns.get_loc('Основной навык кандидата')
COL_SALARY      = df.columns.get_loc('Зарплата кандидата')
COL_EXP         = df.columns.get_loc('Опыт работы кандидата')
COL_SCHED       = df.columns.get_loc('График кандидата')
COL_CITY        = df.columns.get_loc('Регион кандидата')
COL_EMPL        = df.columns.get_loc('Место кандидата') #Удаленная/офис и пр.
COL_POS_PREV    = df.columns.get_loc('Профессия кандидата')
COL_ENG         = df.columns.get_loc('Английский кандидата')
COL_PERS_COMM  = df.columns.get_loc('Примечание кандидата')
# COL_EDU         = df.columns.get_loc('Образование и ВУЗ')
COL_UPDATED     = df.columns.get_loc('Ссылка на резюме')
COL_FILE        = df.columns.get_loc('Файл с резюме')

# Вывод первых нескольких строк датасета для проверки
display(df.head(5))
print(df.shape)

,Тип события,Вакансия,Компания,Кандидат,Дата события,Примечание события,Скилы для вакансии,Основной навык для вакансии,Зарплата для вакансии,Уровень для вакансии,...,Зарплата кандидата,Опыт работы кандидата,График кандидата,Регион кандидата,Место кандидата,Профессия кандидата,Английский кандидата,Примечание кандидата,Ссылка на резюме,Файл с резюме
№,,,,,,,,,,,,,,,,,,,,,
1,Оффер,ML Lead,PersonalInvest - Indata Labs,Бакаев Евгений,01.04.2024,01.04 - на рассмотрение 07.04 - интервью Бакае...,"AWS, Azure, ETL, Machine Learning, python",Machine Learning,8000-12000 в месяц,Head of,...,NaN,15.0,Полный день,Варшава,Удаленная,"Data scientist, Machine Learning Engineer",B2,NaN,NaN,NaN
2,Трудоустроен,QA/Support,Immersive Fox,Лобань Дмитрий,15.04.2024,14.04 - на рассмотрение 21.04 - интервью 17.05...,"Support, Web Testing","Support, Web Testing",1500-2000 в месяц,Middle,...,NaN,3.0,Полный день,Другой,Удаленная,"QA, QA automation, Supporter / Customer service",C1,NaN,NaN,NaN
3,Трудоустроен,digital marketing specialist,Immersive Fox,Куца Татьяна,10.05.2024,10/04 - на рассмотрение 10.05 - оффер 13.05. -...,"Internet Marketing, Marketing, Marketing Analysis",Internet Marketing,4500-5000 в месяц,Senior,...,NaN,12.0,Полный день,Испания,Удаленная,Internet Marketer,C1,NaN,NaN,NaN
4,Отказ,DevOps,SolbegSoft,Мельничук Павел,06.05.2024,"29.04.24 рассмотрение . 06.05.24 К сожалению, ...","Azure, Databases, DevOps, Jira, MSSQL, SQL",Databases,2000-2200 в месяц,Middle,...,2000-2300 в месяц,6.0,NaN,NaN,NaN,DevOps,A2,NaN,NaN,NaN
5,Тестовое задание,Frontend Developer,SolbegSoft,Попов Олег,03.05.2024,20.04.24 рассм. 03.05.24 скинули тестовое,"ES5/ES6, Angular, Javascript, React, Redux, Ty...",Javascript,NaN,Senior,...,NaN,NaN,NaN,NaN,NaN,Frontend developer,NaN,NaN,NaN,NaN


(6212, 26)


In [14]:
from tabulate import tabulate

n = 2  # Индекс в таблице резюме
data = []
for i in range(df.columns.shape[0]):  # Вывод значения каждого столбца
    data.append([f'\033[91m{df.columns[i]}:\033[0m', df.values[n][i]])

print(tabulate(data, tablefmt="grid"))

+------------------------------+----------------------------------------------------------------------+
| Тип события:                 | Трудоустроен                                                         |
+------------------------------+----------------------------------------------------------------------+
| Вакансия:                    | digital marketing specialist                                         |
+------------------------------+----------------------------------------------------------------------+
| Компания:                    | Immersive Fox                                                        |
+------------------------------+----------------------------------------------------------------------+
| Кандидат:                    | Куца Татьяна                                                         |
+------------------------------+----------------------------------------------------------------------+
| Дата события:                | 10.05.2024                     

In [16]:
df.drop(['Примечание кандидата', 'Ссылка на резюме', 'Файл с резюме', 'Компания', 'Дата события', 'Примечание события'], axis=1, inplace=True)

In [17]:
# Проведем анализ датасета
df.shape # Кол-во строк и столбцов в датасете
df.count() # Кол-во непустых записей в каждом из столбцов
df.dtypes # Тип данных в каждом из столбцов
df.info() # Саммари по датасету

<class 'pandas.core.frame.DataFrame'>
Index: 6212 entries, 1 to 6212
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Тип события                  6212 non-null   object 
 1   Вакансия                     6212 non-null   object 
 2   Кандидат                     6212 non-null   object 
 3   Скилы для вакансии           6157 non-null   object 
 4   Основной навык для вакансии  5338 non-null   object 
 5   Зарплата для вакансии        5363 non-null   object 
 6   Уровень для вакансии         6060 non-null   object 
 7   Английский для вакансии      5768 non-null   object 
 8   Место для вакансии           6119 non-null   object 
 9   График для вакансии          6123 non-null   object 
 10  Уровень кандидата            6198 non-null   object 
 11  Скилы кандидата              5887 non-null   object 
 12  Основной навык кандидата     4987 non-null   object 
 13  Зарплата кандидата     

In [18]:
# Удаление дублирующихся строк
df.drop_duplicates(inplace=True)

# Удаление строк, содержащих NaN
df.dropna(inplace=True)

# Проверка результата
print(f"Размер таблицы после удаления дубликатов и строк с NaN: {df.shape}")
df.head()

Размер таблицы после удаления дубликатов и строк с NaN: (1396, 20)


,Тип события,Вакансия,Кандидат,Скилы для вакансии,Основной навык для вакансии,Зарплата для вакансии,Уровень для вакансии,Английский для вакансии,Место для вакансии,График для вакансии,Уровень кандидата,Скилы кандидата,Основной навык кандидата,Зарплата кандидата,Опыт работы кандидата,График кандидата,Регион кандидата,Место кандидата,Профессия кандидата,Английский кандидата
№,,,,,,,,,,,,,,,,,,,,
16,Самоотказ,Backend Developer (Golang),Сачук Алина,"ClickHouse, Docker, Go/Golang, MongoDB, NoSQL,...",Go/Golang,3000-4000 в месяц,Senior,A2,"Релокейт, Удаленная",Полный день,Middle,Go/Golang,Go/Golang,4000-4000 в месяц,3.0,Полный день,Другие страны,"Офис, Удаленная",Backend developer,B1
19,Отказ,Backend Developer (Golang),Тяпин Алекс,"ClickHouse, Docker, Go/Golang, MongoDB, NoSQL,...",Go/Golang,3000-4000 в месяц,Senior,A2,"Релокейт, Удаленная",Полный день,Senior,"Go/Golang, python",Go/Golang,4000-4000 в месяц,5.0,Полный день,Грузия,Удаленная,Backend developer,B1
20,Отказ,Backend Developer (Golang),Бабло Никита,"ClickHouse, Docker, Go/Golang, MongoDB, NoSQL,...",Go/Golang,3000-4000 в месяц,Senior,A2,"Релокейт, Удаленная",Полный день,Middle,"Ajax, Bootstrap, HTML / CSS, Javascript, Larav...",PHP,2000-2000 в месяц,7.0,Полный день,Минск,"Релокейт, Удаленная","Backend developer, Web-developer",C1
23,Отказ,Backend Developer (Golang),Бабло Никита,"ClickHouse, Docker, Go/Golang, MongoDB, NoSQL,...",Go/Golang,3000-4000 в месяц,Senior,A2,"Релокейт, Удаленная",Полный день,Senior,Go/Golang,Go/Golang,6000-6000 в месяц,6.0,Полный день,Другие страны,"Офис, Удаленная",Backend developer,B2
26,Самоотказ,Backend Developer (Golang),Иванов Сергей,"ClickHouse, Docker, Go/Golang, MongoDB, NoSQL,...",Go/Golang,3000-4000 в месяц,Senior,A2,"Релокейт, Удаленная",Полный день,Lead,Go/Golang,Go/Golang,6000-6000 в месяц,10.0,Полный день,Другие страны,Удаленная,Backend developer,B2


In [19]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1396 entries, 16 to 5697
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Тип события                  1396 non-null   object 
 1   Вакансия                     1396 non-null   object 
 2   Кандидат                     1396 non-null   object 
 3   Скилы для вакансии           1396 non-null   object 
 4   Основной навык для вакансии  1396 non-null   object 
 5   Зарплата для вакансии        1396 non-null   object 
 6   Уровень для вакансии         1396 non-null   object 
 7   Английский для вакансии      1396 non-null   object 
 8   Место для вакансии           1396 non-null   object 
 9   График для вакансии          1396 non-null   object 
 10  Уровень кандидата            1396 non-null   object 
 11  Скилы кандидата              1396 non-null   object 
 12  Основной навык кандидата     1396 non-null   object 
 13  Зарплата кандидата    

In [21]:
# Вывод уникальных значений в столбце 'Тип события'
unique_COL_IVENT_TYPE = df['Тип события'].unique()
print(unique_COL_IVENT_TYPE)
print(len(unique_COL_IVENT_TYPE))

['Самоотказ' 'Отказ' 'Оффер' 'Самоотказ до интервью' 'Тестовое задание'
 'Трудоустроен' 'Отказ от оффера в пользу другой компании'
 'Отказ от оффера (контр-оффер)' 'Не прошел ИС']
9


In [ ]:
### Параметрические данные для функций разбора ###

# Курсы валют для зарплат
currency_rate = {
    'usd'    : 90.3532,     # 1 USD = 90.3532 RUB[^1^][1]
    'kzt'    : 0.200,       # 1 KZT = 0.200 RUB
    'грн'    : 2.42,        # 1 UAH = 2.42 RUB
    'белруб' : 27.5,        # 1 BYN = 27.5 RUB[^1^][1]
    'eur'    : 98.0,        # 1 EUR = 98.0 RUB
    'kgs'    : 0.96,        # 1 KGS = 0.96 RUB
    'сум'    : 0.0076,      # 1 UZS = 0.0076 RUB
    'azn'    : 53.0         # 1 AZN = 53.0 RUB
}


# Списки и словари для разбиения на классы
# Для ускорения работы добавлен счетчик классов, который будет вычислен ниже

# Список порогов возраста
age_class = [0, [18, 23, 28, 33, 38, 43, 48, 53, 58, 63]]

# Список порогов опыта работы в месяцах
experience_class = [0, [7, 13, 25, 37, 61, 97, 121, 157, 193, 241]]

# Классы городов
city_class = [0,
              {'москва'          : 0,
               'санкт-петербург' : 1,
               'новосибирск'     : 2,
               'екатеринбург'    : 2,
               'нижний новгород' : 2,
               'казань'          : 2,
               'челябинск'       : 2,
               'омск'            : 2,
               'самара'          : 2,
               'ростов-на-дону'  : 2,
               'уфа'             : 2,
               'красноярск'      : 2,
               'пермь'           : 2,
               'воронеж'         : 2,
               'волгоград'       : 2,
               'прочие города'   : 3
              }]

# Классы занятости
employment_class = [0,
                    {'стажировка'          : 0,
                     'частичная занятость' : 1,
                     'проектная работа'    : 2,
                     'полная занятость'    : 3
                    }]

# Классы графика работы
schedule_class = [0,
                  {'гибкий график'         : 0,
                   'полный день'           : 1,
                   'сменный график'        : 2,
                   'удаленная работа'      : 3
                  }]

# Классы образования
education_class = [0,
                   {'высшее образование'   : 0,
                    'higher education'     : 0,
                    'среднее специальное'  : 1,
                    'неоконченное высшее'  : 2,
                    'среднее образование'  : 3
                   }]
